## Basic Implementation Based on Most Important Words
---

**This implementation uses frequency distribution of words to rank each sentence and select the top sentences**

### Improvements (To Do)
1. Based on pos tagging we can give preference to specific type of pos like nouns etc.

In [61]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import string

class Summarizer:
    stopwords = stopwords.words("english")

    def __init__(self,min_freq=0.1,max_freq=0.9):
        self._min_freq = min_freq
        self._max_freq = max_freq
        
    def get_useful_bag_of_words(self,text):
        bag_of_words = [w for w in word_tokenize(text) if w not in (self.stopwords + list(string.punctuation))]
        return bag_of_words
    
    def score_sent(self,sent,word_rank_list):
        score = 0
        for w in word_tokenize(sent.lower()):
            if w in word_rank_list.keys():
                score += word_rank_list[w]
        return score
    
    def reorder_sents(self,sents,text):
        sents.sort( lambda s1, s2: text.find(s1) - text.find(s2) )
        return sents
    
    def get_summary(self,text,sent_num):
        sane_text = unicode(text, "utf-8")
        
        word_freq_dist = FreqDist(self.get_useful_bag_of_words(sane_text.lower()))
        
        word_ranks = {}
        for word in word_freq_dist:
            word_ranks[word] = word_freq_dist.freq(word)
            
        sents = sent_tokenize(sane_text)
        
        if sent_num > len(sents):
            print "text already summarized !!!"
            return
        
        full_output = []
        for sent in sents:
            temp = {}
            temp['sent'] = sent
            temp['score'] = self.score_sent(sent,word_ranks)
            full_output.append(temp)
            
        sorted_full_output = sorted(full_output, key=lambda k: k['score'], reverse=True)
        top_output_sents = [s['sent'] for s in sorted_full_output[:sent_num]]
        reordered_sents = self.reorder_sents(top_output_sents,sane_text)
        return "".join(reordered_sents)
    

In [62]:
test_summarizer = Summarizer()

test_sent = """To Sherlock Holmes she is always the woman. I have
seldom heard him mention her under any other name. In his eyes she
eclipses and predominates the whole of her sex. It was not that he
felt any emotion akin to love for Irene Adler. All emotions, and that
one particularly, were abhorrent to his cold, precise but admirably
balanced mind. He was, I take it, the most perfect reasoning and
observing machine that the world has seen, but as a lover he would
have placed himself in a false position. He never spoke of the softer
passions, save with a gibe and a sneer. They were admirable things for
the observer-excellent for drawing the veil from men’s motives and
actions. But for the trained reasoner to admit such intrusions into
his own delicate and finely adjusted temperament was to introduce a
distracting factor which might throw a doubt upon all his mental
results. Grit in a sensitive instrument, or a crack in one of his own
high-power lenses, would not be more disturbing than a strong emotion
in a nature such as his. And yet there was but one woman to him, and
that woman was the late Irene Adler, of dubious and questionable
memory.
"""

summary = " ".join(test_summarizer.get_summary(test_sent,5).split())

print summary

All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind.He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position.But for the trained reasoner to admit such intrusions into his own delicate and finely adjusted temperament was to introduce a distracting factor which might throw a doubt upon all his mental results.Grit in a sensitive instrument, or a crack in one of his own high-power lenses, would not be more disturbing than a strong emotion in a nature such as his.And yet there was but one woman to him, and that woman was the late Irene Adler, of dubious and questionable memory.
